In [19]:
!pip install opencv-contrib-python==3.4.0.12 # otherwise you'd get an error message saying this when you try to use SIFT
                                   # This algorithm is patented and is excluded in this configuration; 
                                   # Set OPENCV_ENABLE_NONFREE CMake option and rebuild the library in 
                                   # function 'cv::xfeatures2d::SIFT::create'

In [20]:
import numpy as np
from scipy import ndimage, signal
from scipy.ndimage import gaussian_filter
from copy import deepcopy
import matplotlib.pyplot as plt

import random
import math
import os

!pip install opencv-python
import cv2

import plotly.graph_objects as go

from google.colab.patches import cv2_imshow

In [21]:
GAUSSIAN_BLUR_SIGMA=1

In [22]:
# Modified code from assignment
def compute_gaussian_subsample(image:np.ndarray, factor:int=2) -> np.ndarray:
    """
    :param np.ndarray image: The image
    :param int factor: factor to scale down by
    :returns: A copy of the image, scaled down and blurred using gaussian filter
    :rtype: np.ndarray
    """

    temp_image = gaussian_filter(deepcopy(image), sigma=GAUSSIAN_BLUR_SIGMA)

    len_x = temp_image.shape[0]
    len_y = temp_image.shape[1]
    half_len_x = int(len_x/2)
    half_len_y = int(len_y/2)

    # Subsample every other pixel
    temp_image = deepcopy(temp_image[::factor,::factor])

    # Gaussain blur
    temp_image = gaussian_filter(temp_image, sigma=GAUSSIAN_BLUR_SIGMA)

    return temp_image

In [23]:
def get_images(path0, path1, factor=2):
  """
  :param string path0: Path to image 0
  :param string path1: Path to image 1
  :param int factor: factor to scale down by
  :returns: The two images, scaled down by factor
  :rtype: np.ndarray, np.ndarray
  """
  img0 = cv2.imread(path0, cv2.IMREAD_GRAYSCALE)
  img1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)

  # subsample because current imgs too big
  img0 = compute_gaussian_subsample(img0,factor)
  img1 = compute_gaussian_subsample(img1,factor)

  #plt.subplot(1, 2, 1)
  #plt.imshow(img0, cmap='gray')
  #plt.subplot(1, 2, 2)
  #plt.imshow(img1, cmap='gray')

  return img0, img1

In [24]:
# Modified SIFT code from assignment
def match_points(img1, img2, match_method='sift', ratio_param=0.75):
  """
  Adapted from lab code for assignment 3.
  :param np.ndarray img1: the left image
  :param np.ndarray img2: the right image
  :param string match_method: method for feature matching
  :param float ratio_param: ratio to use in ratio test
  :returns: The matching points from the left and right images
  :rtype: np.ndarray, np.ndarray
  """
  if match_method == 'orb' or match_method == 'ORB':
    orb = cv2.ORB_create(nfeatures=1000)
    kp1, desc1 = orb.detectAndCompute(img1, None)
    kp2, desc2 = orb.detectAndCompute(img2, None)
  elif match_method == 'surf' or match_method == 'SURF':
    surf = cv2.xfeatures2d.SURF_create()
    kp1, desc1 = surf.detectAndCompute(img1, None)
    kp2, desc2 = surf.detectAndCompute(img2, None)
  else:
    sift = cv2.xfeatures2d.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(img1, None)
    kp2, desc2 = sift.detectAndCompute(img2, None)

  bf = cv2.BFMatcher()
  matches = bf.knnMatch(desc1, desc2, k=2) # k=2 means find the top two matchs for each query descriptor

  # Apply ratio test (as per David Lowe's SIFT paper: compare the best match with the 2nd best match_
  good_matches = []
  good_matches_without_list = []
  for m,n in matches:
      if m.distance < ratio_param*n.distance: # only accept matchs that are considerably better than the 2nd best match
          good_matches.append([m])
          good_matches_without_list.append(m) # this is to simplify finding a homography later

  src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches_without_list ]).reshape(-1,1,2)
  dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches_without_list ]).reshape(-1,1,2)

  return src_pts, dst_pts

In [25]:
def extract_calibration_vals(file):
  """
  Code to extract parameters from calibration files
  :param string file: filepath to extract from
  :returns: A dictionary containing the calibration parameters
  :rtype: dict
  """

  calib_vals = {}
  with open(file) as f:
    lines = f.readlines() # list containing lines of file
    for line in lines:
      line_split = line.split('=')
      if line_split[0] == 'cam0' or line_split[0] == 'cam1':
        matrix = line_split[1].strip('[]')
        matrix = matrix.split(';')
        matrix_vals = [[float(val.strip('[]')) for val in matrix_line.split()] for matrix_line in matrix]
        calib_vals[line_split[0]] = np.array(matrix_vals)
      else:
        calib_vals[line_split[0]] = float(line_split[1].rstrip())
  return calib_vals

#extract_calibration_vals('calib.txt')

In [26]:
# remove points that are n stdev away from the mean
def remove_outliers(world_coord, num_std=3):
  """
  Removes points that are more than num_std standard deviations away from the mean in any direction
  This is to prevent bad matches from skewing the resulting 3d plot
  :param np.ndarray world_coord: array of world coordinate points
  :param float num_std: number of standard deviations to act as a cutoff point for outliers
  :returns: The inliers that are within num_std standard deviations
  :rtype: np.ndarray
  """

  mean_coord = np.array([np.mean(world_coord[:,0]), np.mean(world_coord[:,1]), np.mean(world_coord[:,2])])
  stdev_coord = np.array([np.std(world_coord[:,0]), np.std(world_coord[:,1]), np.std(world_coord[:,2])])

  world_coord = world_coord[(world_coord[:,0] > mean_coord[0] - num_std*stdev_coord[0]) & (world_coord[:,0] < mean_coord[0] + num_std*stdev_coord[0])]
  world_coord = world_coord[(world_coord[:,1] > mean_coord[1] - num_std*stdev_coord[1]) & (world_coord[:,1] < mean_coord[1] + num_std*stdev_coord[1])]
  world_coord = world_coord[(world_coord[:,2] > mean_coord[2] - num_std*stdev_coord[2]) & (world_coord[:,2] < mean_coord[2] + num_std*stdev_coord[2])]

  return world_coord

In [27]:
# Get the 3D coordinates of the matches
def get_world_coord(path0, path1, folder, match_method='sift', ratio_param=0.75):
  """
  Calculate the world coordinates for feature matches between the images in path0 and path1.
  Uses the method described in CSC420 lecture in the Stereo slides.
  :param string path0: the left image path
  :param string path1: the right image path
  :param string folder: folder location
  :param string match_method: method to use for feature matching
  :param float ratio_param: ratio to use for ratio test
  :returns: The world coordinates of the matching points in the images in path0, path1
  :rtype: np.ndarray
  """

  factor = 2
  path0 = os.path.join(folder, path0)
  path1 = os.path.join(folder, path1)
  img0, img1 = get_images(path0, path1, factor)
  rgb0 = cv2.imread(path0)
  # Extract matching points from the imgs
  img0_pts, img1_pts = match_points(img0, img1, match_method, ratio_param)

  # Acquire camera information
  calib_vals = extract_calibration_vals(os.path.join(folder,'calib.txt'))
  K0 = calib_vals['cam0']
  K1 = calib_vals['cam1']
  f = K0[0,0]
  T = calib_vals['doffs']
  px0 = K0[0,2]
  px1 = K1[0,2]
  py0 = K0[1,2]
  py1 = K1[1,2]

  img0_pts = np.array(img0_pts)
  img1_pts = np.array(img1_pts)

  img0_pts = img0_pts.reshape((img0_pts.shape[0],img0_pts.shape[2]))
  img1_pts = img1_pts.reshape((img1_pts.shape[0],img1_pts.shape[2]))

  # Calculate world coordinates per pixel
  num_pts = len(img0_pts)
  world_coord = np.zeros((num_pts,6))
  for idx, pt in enumerate(img0_pts):
    # Calculate depth using: z = fT/(xr-xl)
    z = f*T/(((pt[0])-(img1_pts[idx,0]))*factor)
    world_coord[idx,2] = z

    # x = fX/z + px, solve for X where X is world coordinate of x in the img
    # X = (x-px)z/f
    x0 = (pt[0]-px0)*factor*z/f
    x1 = (img1_pts[idx,0]-px1)*factor*z/f
    world_coord[idx,0] = -x0

    # do same for Y
    y0 = (pt[1]-py0)*factor*z/f
    y1 = (img1_pts[idx,1]-py1)*factor*z/f
    world_coord[idx,1] = y0

    # add colour
    world_coord[idx,3:] = rgb0[int(pt[1])*factor,int(pt[0])*factor,:]

  return world_coord

# plot 3d point cloud of matches via plotly
def plot_pointCloud(pc, path='plot.html'):
    '''
    Uses code from Assignment 4.
    plots the Nx6 point cloud pc in 3D
    assumes (1,0,0), (0,1,0), (0,0,-1) as basis
    :param np.ndarray pc: the points in world coordinates
    :param string path: path to save plot to
    :returns: None
    :rtype: None
    '''
    fig = go.Figure(data=[go.Scatter3d(
        x=pc[:, 0],
        y=pc[:, 1],
        z=-pc[:, 2],
        mode='markers',
        marker=dict(
            size=2,
            color=pc[:, 3:][..., ::-1],
            opacity=0.8
        )
    )])
    fig.write_html(path)
    fig.show()



In [28]:
def test_imgs(path0, path1, folder='', match_method='sift'):
  """
  Shows the matching points on a 2d plane in 3d space for visualization purposes.
  :param string path0: the left image path
  :param string path1: the right image path
  :param string folder: folder location
  :param string match_method: method to use for feature matching

  :returns: None
  :rtype: None
  """
  factor = 2
  path0 = os.path.join(folder, path0)
  path1 = os.path.join(folder, path1)
  img0, img1 = get_images(path0, path1, factor)
  rgb0 = cv2.imread(path0)

  # Extract matching points from the imgs
  img0_pts, img1_pts = match_points(img0, img1, match_method)

  img0_pts = np.array(img0_pts)
  img1_pts = np.array(img1_pts)
  img0_pts = img0_pts.reshape((img0_pts.shape[0],img0_pts.shape[2]))
  img1_pts = img1_pts.reshape((img1_pts.shape[0],img1_pts.shape[2]))

  num_pts = len(img0_pts)
  test_coord = np.zeros((num_pts,6))
  for idx,pt in enumerate(img0_pts):
    test_coord[idx,0] = -pt[0]*factor
    test_coord[idx,1] = pt[1]*factor
    test_coord[idx,2] = 5
    test_coord[idx,3:] = rgb0[int(pt[1])*factor,int(pt[0])*factor,:]

  plot_pointCloud(test_coord)


In [29]:
def run(img0='im0.png', img1='im1.png', folder='', match_method='sift', ratio=0.75, outlier_bound=2, include_2d=True):
  """
  Runs the methods to calculate world coordinates, remove outliers, and display points.
  :param string img0: the left image name
  :param string img11: the right image name
  :param string folder: folder location
  :param string match_method: method to use for feature matching
  :param float ratio_param: ratio to use for ratio test
  :param float outlier_bound: number of standard deviation away from mean for point to be considered an outlier
  :param bool include_2d: whether to include a 3d plot showing the matches on a 2d plane
  :returns: None
  :rtype: None
  """
  if include_2d:
    # if you want to see the 2d point matches
    test_imgs(img0,img1,folder,match_method)
  world_coord = get_world_coord(img0, img1, folder, match_method, ratio)
  world_coord = remove_outliers(world_coord, outlier_bound)
  plot_pointCloud(world_coord)


In [30]:
!unzip Imgs.zip

Archive:  Imgs.zip
replace Imgs/Bicycle/calib.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [31]:
def run_folder(pic):
  """
  Runs the main run function for a particular image.
  :param int pic: index of pic_list to use for running parallel stereo pipeline
  :returns: None
  :rtype: None
  """

  pic_list = ['Bicycle', 'JadePlant', 'Mask', 'Recycling', 'Storage']
  folder_name = os.path.join('Imgs', pic_list[pic])

  run('im0.png', 'im1.png', folder=folder_name, match_method = 'sift', ratio=0.75, outlier_bound=2, include_2d=False)

# The argument is the index of the folder in pic_list that you want to analyse
run_folder(0)